Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [3]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

,player,starter,percentage
0,Ochoa,1,90.0
1,Lovato,1,80.0
2,Gyomber,1,80.0
3,Pirola,1,80.0
4,Mazzocchi,1,80.0
...,...,...,...
471,Pisilli,0,10.0
472,Aouar,0,55.0
473,El Shaarawy,0,55.0
474,Belotti,0,60.0


In [4]:
probables = probables.set_index('player')

In [5]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

,player1,player2,percentage
0,Ikwuemesi,Botheim,55.0
1,Jovane,Kastanos,55.0
2,Okoli,Monterisi,55.0
3,Caso,Baez,55.0
4,Vasquez,Martin,60.0
5,Malinovskyi,De Winter,60.0
6,Kjaer,Thiaw,60.0
7,Rafael Leao,Chukwueze,60.0
8,Giroud,Jovic,60.0
9,Bonazzoli,Djuric,60.0


In [6]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

,starter,percentage
player,,
Ochoa,1,90.0
Lovato,1,80.0
Gyomber,1,80.0
Pirola,1,80.0
Mazzocchi,1,80.0
...,...,...
Pisilli,0,10.0
Aouar,0.4,55.0
El Shaarawy,0,55.0


In [7]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   